In [53]:
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd
from pandas import read_csv
from sklearn.utils import shuffle
from datetime import datetime

In [54]:
df_data = pd.read_pickle('CleanedDataWithIds.pkl') 
df_data = shuffle(df_data)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [55]:
df_data.head()

,elementId,timeStamp,type,siteName,date,clientX,clientY,session,height,width,browser,key,sitename
17447,inputTemperature,0,keydown,0,2022-10-19T20:48:00.484Z,NaN,NaN,338,669,390,NaN,Backspace,NaN
14603,inputDifficultQuestion,121161,focus,1,2022-10-19T12:34:54.576Z,NaN,NaN,712,970,954,Firefox,NaN,NaN
35162,inputHand,15046.000000000002,focus,0,2022-10-22T15:34:16.822Z,NaN,NaN,737,593,375,Safari,NaN,NaN
6484,inputCountry,5606.799999952316,keydown,0,2022-10-18T14:30:44.918Z,NaN,NaN,369,962,1724,NaN,Backspace,NaN
14381,inputHouse,11811,blur,0,2022-10-19T12:32:17.859Z,NaN,NaN,508,970,954,Firefox,NaN,NaN


# Hier kommt DataFrame sortiert nach Session

In [56]:
df_replaced_together = df_data.sort_values(by=['session','date'], ascending=[False, True])

In [57]:
df_replaced_together['real_date'] = df_replaced_together['date'].astype('datetime64[ns]')
#df_replaced_together['real_date'] = (df_replaced_together['real_date'] - df_replaced_together['real_date'].min())  / np.timedelta64(1,'D')
df_replaced_together = df_replaced_together.drop(['date', 'clientX', 'clientY', 'sitename', 'key', 'timeStamp'], axis=1)
df_replaced_together = df_replaced_together.loc[df_replaced_together["type"] !="scroll"]

In [58]:
#remove'clickHere', 'buttonYes' 'alibiClick'
df_replaced_together = df_replaced_together.loc[~(df_replaced_together['elementId'] =='clickHere')]
df_replaced_together = df_replaced_together.loc[~(df_replaced_together['elementId'] =='alibiClick')]
df_replaced_together = df_replaced_together.loc[~(df_replaced_together['elementId'] =='buttonYes')]

                    elementId       type  siteName  session height width  \
2388               inputHobby  mouseover         1      997    520   360   
2389               inputHobby      focus         1      997    520   360   
2392               inputHobby   mouseout         1      997    277   360   
2393               inputHobby       blur         1      997    277   360   
2398                   window       blur         1      997    520   360   
2401                   window      focus         1      997    616   360   
21337            inputCountry  mouseover         0      995    827   412   
21338            inputCountry      focus         0      995    827   412   
21340            inputCountry       blur         0      995    478   412   
21339            inputCountry      focus         0      995    478   412   
21359            inputCountry   mouseout         0      995    478   412   
21360              inputHouse  mouseover         0      995    478   412   
21361       

In [59]:
#get all new id numbers 
sessionId = df_replaced_together['session']
df_replaced_together['is_duplicated'] = df_replaced_together.duplicated('session')
#86 ids auf Seite Einfach
numbers1 = set(sessionId)
numbers = list(numbers1)

print(numbers)

[512, 518, 519, 520, 538, 555, 588, 589, 591, 104, 622, 128, 134, 647, 137, 650, 651, 652, 142, 656, 149, 662, 683, 176, 689, 693, 186, 188, 711, 712, 199, 715, 717, 728, 218, 221, 222, 737, 748, 237, 240, 754, 243, 244, 242, 246, 759, 760, 247, 762, 253, 766, 255, 259, 772, 268, 786, 787, 277, 280, 290, 298, 813, 820, 309, 829, 837, 838, 842, 330, 846, 338, 853, 342, 856, 347, 349, 350, 353, 871, 874, 363, 369, 372, 890, 384, 898, 901, 904, 393, 395, 914, 917, 410, 925, 926, 927, 416, 929, 421, 937, 427, 952, 441, 955, 444, 958, 962, 964, 966, 455, 456, 454, 975, 464, 465, 463, 981, 474, 475, 476, 990, 991, 480, 995, 997, 487, 495, 503, 504, 505, 508]


In [60]:
hey = pd.DataFrame(numbers, columns=["session"])
arr1 = []
arr2 = []
arr3 = []
arr4 = []
arr5 = []
arr6 = []
arr7 = []
arr8 = []
arrDate = []
arrHeight = []
arrWidth = []
arrSiteName = []

for i in range(len(numbers)):
    new = df_replaced_together.loc[df_replaced_together['session'] == numbers[i]]
    #get first and last entry of new dataframe
    #first = new['real_date'].iloc[0]
    date = (new['real_date'].iloc[-1] - (new['real_date'].iloc[0])).total_seconds()
    arrDate.append(date)
    height = new['height'].iloc[0]
    arrHeight.append(height)
    width = new['width'].iloc[0]
    arrWidth.append(width)
    siteName = new['siteName'].iloc[0]
    arrSiteName.append(siteName)
    #print(new)
    #print(date)
    # last = new.iloc[-1]
    new2 = (len(new.loc[new['type']=='mouseover']))
    arr2.append(new2)
    new3 = (len(new.loc[new['type'] =='mouseout']))
    arr3.append(new3)
    new4 = (len(new.loc[new['type'] =='click']))
    arr4.append(new4)
    new5 = (len(new.loc[new['type'] =='blur']))
    arr5.append(new5)
    new6 = (len(new.loc[new['type'] =='focus']))
    arr6.append(new6)
    new7 = (len(new.loc[new['type'] =='keydown']))
    arr7.append(new7)

hey['count_mouseover'] = arr2
hey['count_mouseout'] = arr3
hey['count_click'] = arr4
hey['count_blur'] = arr5
hey['count_focus'] = arr6
hey['count_keydown'] = arr7
hey['duration'] = arrDate
hey['height'] = arrHeight
hey['width'] = arrWidth
hey['siteName'] = arrSiteName

# for i in range(len(types)):
#     hey['type_' + str(types[i])] = arr



#hey['duration'] = df_replaced_together['date']


In [61]:
hey = hey.loc[hey['duration'] < 2000]

In [62]:
hey.head()

,session,count_mouseover,count_mouseout,count_click,count_blur,count_focus,count_keydown,duration,height,width,siteName
0,512,7,7,0,7,7,4,18.157,664,390,0
1,518,70,70,25,31,32,13,459.177,667,1440,1
2,519,6,6,0,7,7,0,18.090,775,412,0
3,520,39,39,3,14,13,29,232.459,1217,1440,1
4,538,8,8,4,17,15,20,213.904,664,390,1


In [63]:
hey.to_pickle('SortedBySessions.pkl')

Test

In [64]:
#clientX und clientY (rot ist schwer, grün ist einfach)
df_replaced_together2 = df_data.sort_values(by=['session','date'], ascending=[False, True])
df_replaced_together2['real_date'] = df_replaced_together2['date'].astype('datetime64[ns]')
df_replaced_together2 = df_replaced_together2.drop(['date', 'sitename', 'key', 'timeStamp'], axis=1)

df_mouseover = df_replaced_together2.loc[df_replaced_together2['type']=='mouseover']
df_mouseout = df_replaced_together2.loc[df_replaced_together2['type']=='mouseout']
df_mouse = pd.concat([df_mouseover, df_mouseout])


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

session = df_mouse.loc[df_mouse['session'] == 926]
session = session.sort_values(by=['real_date'], ascending=[True])

print(session)

# plt.gca().invert_yaxis()

# plt.scatter(x_schwer,y_schwer, c='red')
# plt.scatter(x_einfach,y_einfach,c='green')

                    elementId       type  siteName clientX clientY  session  \
16293        inputTemperature  mouseover         1     863     452      926   
16294        inputTemperature   mouseout         1     863     407      926   
16295              inputHobby  mouseover         1     882     316      926   
16296              inputHobby   mouseout         1     890     282      926   
16297              inputHobby  mouseover         1     958     286      926   
16299              inputHobby   mouseout         1     967     301      926   
16308           inputCancelor  mouseover         1    1729     443      926   
16309           inputCancelor   mouseout         1    1345     426      926   
16310              inputHobby  mouseover         1    1029     465      926   
16312              inputHobby   mouseout         1    1009     509      926   
16313        inputTemperature  mouseover         1    1091     616      926   
16314        inputTemperature   mouseout         1  